In [1]:
import pymeshlab

import subprocess as sb
import os
# p = subprocess.run("cp", "dirA/file", "dirB")
from ipywidgets import IntProgress
from IPython.display import display

In [32]:
# reference: https://github.com/vonunwerth/MeshLabInertiaToURDF/blob/master/main.py
import numpy as np


def calculate_inertial_tag(file_name=None, density=1, pr=8, scale_factor=100):
    if 'robotiq' in file_name and 'base_link' not in file_name:
        measure_scale = 1000
    else:
        measure_scale = 1
    ms = pymeshlab.MeshSet()

    if file_name is None:
        print('Please put the input file to the same folder as this script and type in the full name of your file.')
        file_name = input()
    ms.load_new_mesh(file_name)

#     if mass < 0:
#         print('Please type the density of your object in kg/m^3')
#         mass = float(input())

#     geom = ms.apply_filter('Compute Geometric Measures')
    geom = ms.get_geometric_measures()
    com = geom['barycenter']
    com = np.array(com) / measure_scale
    
    # apply the filter to remove duplicate faces
#     ms.apply_filter('meshing_remove_duplicate_faces')
    ms.meshing_remove_duplicate_faces()
    
    ms.compute_matrix_from_scaling_or_normalization(axisx=scale_factor, axisy=scale_factor, axisz=scale_factor)

    ms.generate_convex_hull()  # TODO only if object is not watertight

    geom = ms.get_geometric_measures()
    volume = geom['mesh_volume']
#     tensor = geom['inertia_tensor'] * density

    tensor = geom['inertia_tensor'] / pow(scale_factor, 5) * density
#     tensor = geom['inertia_tensor'] / pow(scale_factor, 2) * density
    mass = density * volume / pow(scale_factor, 3)
    
    tensor =tensor / pow(measure_scale, 5)
    mass = mass / pow(measure_scale, 3)

    intertial_xml = f'<inertial>\n  <origin xyz="{com[0]:.{pr}f} {com[1]:.{pr}f} {com[2]:.{pr}f}"/>\n  <mass value="{mass:.{pr}f}"/>\n  <inertia ixx="{tensor[0, 0]:.{pr}f}" ixy="{tensor[1, 0]:.{pr}f}" ixz="{tensor[2, 0]:.{pr}f}" iyy="{tensor[1, 1]:.{pr}f}" iyz="{tensor[1, 2]:.{pr}f}" izz="{tensor[2, 2]:.{pr}f}"/>\n</inertial>'
    print(intertial_xml)



In [33]:
def obtain_inertia(dir_name):
    files = os.listdir(dir_name)
    # for each file, check if it's STL, and convert only if so

    f = IntProgress(min=0, max=len(files)) # instantiate the bar
    display(f) # display the bar
    
    for file in files:
        ext = file.split('.')[-1]
        name = file.split('.')[0]
        if ext.lower() == 'obj':
            # convert
            print('name: ', name)
            calculate_inertial_tag(file_name=os.path.join(dir_name, file), density=1, pr=8, scale_factor=100)
    

In [34]:
base_dir_name = '/home/yinglong/Documents/research/task_motion_planning/infrastructure/motoman_ws/src/pracsys-motoman/motoman/motoman_config/updated_obj_urdf'
for name1 in ['onrobot_vgc10', 'robotiq_85', 'sda10f_description/meshes/sda10f']:
    for name2 in ['collision']:
        dir_name = os.path.join(base_dir_name, name1, name2)
        obtain_inertia(dir_name)


IntProgress(value=0, max=4)

name:  base_link
<inertial>
  <origin xyz="0.04835756 -0.00001145 -0.00005143"/>
  <mass value="0.00081386"/>
  <inertia ixx="0.00000110" ixy="0.00000000" ixz="0.00000000" iyy="0.00000115" iyz="-0.00000000" izz="0.00000115"/>
</inertial>
name:  suction_cup
<inertial>
  <origin xyz="0.05012417 -0.00023699 -0.00003326"/>
  <mass value="0.00005331"/>
  <inertia ixx="0.00000001" ixy="0.00000000" ixz="0.00000000" iyy="0.00000003" iyz="-0.00000000" izz="0.00000003"/>
</inertial>


IntProgress(value=0, max=12)

name:  robotiq_arg2f_85_base_link
<inertial>
  <origin xyz="0.04288860 -0.00153989 0.00016929"/>
  <mass value="0.00031045"/>
  <inertia ixx="0.00000020" ixy="-0.00000000" ixz="0.00000000" iyy="0.00000024" iyz="0.00000000" izz="0.00000028"/>
</inertial>
name:  robotiq_arg2f_85_outer_knuckle
<inertial>
  <origin xyz="0.00000000 0.01959428 0.00167623"/>
  <mass value="0.00001294"/>
  <inertia ixx="0.00000000" ixy="0.00000000" ixz="0.00000000" iyy="0.00000000" iyz="-0.00000000" izz="0.00000000"/>
</inertial>
name:  robotiq_arg2f_85_inner_knuckle
<inertial>
  <origin xyz="-0.00000000 0.02072093 0.02448419"/>
  <mass value="0.00002731"/>
  <inertia ixx="0.00000001" ixy="0.00000000" ixz="0.00000000" iyy="0.00000001" iyz="-0.00000000" izz="0.00000001"/>
</inertial>
name:  robotiq_arg2f_85_outer_finger
<inertial>
  <origin xyz="0.00000000 0.00303447 0.02345961"/>
  <mass value="0.00001778"/>
  <inertia ixx="0.00000001" ixy="-0.00000000" ixz="-0.00000000" iyy="0.00000001" iyz="-0.00000000" izz=

IntProgress(value=0, max=35)

name:  motoman_axis_b1_remeshed
<inertial>
  <origin xyz="0.29769593 -0.00048507 0.07556016"/>
  <mass value="0.02557033"/>
  <inertia ixx="0.00027808" ixy="-0.00000000" ixz="-0.00009991" iyy="0.00050765" iyz="0.00000000" izz="0.00047731"/>
</inertial>
name:  baxter_suction2
<inertial>
  <origin xyz="0.03122749 0.00233125 -0.00130293"/>
  <mass value="0.00015032"/>
  <inertia ixx="0.00000007" ixy="0.00000000" ixz="-0.00000000" iyy="0.00000008" iyz="-0.00000000" izz="0.00000007"/>
</inertial>
name:  motoman_axis_u
<inertial>
  <origin xyz="-0.01491214 0.04719666 -0.00082991"/>
  <mass value="0.00301532"/>
  <inertia ixx="0.00002075" ixy="-0.00000241" ixz="0.00000001" iyy="0.00000557" iyz="-0.00000003" izz="0.00002043"/>
</inertial>
name:  motoman_axis_s
<inertial>
  <origin xyz="-0.19899234 -0.05062831 -0.00088638"/>
  <mass value="0.00736469"/>
  <inertia ixx="0.00002919" ixy="-0.00000467" ixz="-0.00000000" iyy="0.00006572" iyz="0.00000002" izz="0.00006631"/>
</inertial>
name:  motoman

In [ ]:
ms = pymeshlab.MeshSet()
print(pymeshlab.print_filter_list())